# Sidebar

> Navigation menu components for settings sidebar

In [ ]:
#| default_exp components.sidebar

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
from typing import Dict, Any, Optional, List
from fasthtml.common import *
from cjm_fasthtml_daisyui.components.navigation.menu import menu, menu_title, menu_modifiers
from cjm_fasthtml_daisyui.components.layout.divider import divider
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, items, justify
)
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import  w, h, min_h
from cjm_fasthtml_tailwind.utilities.typography import font_weight
from cjm_fasthtml_tailwind.utilities.layout import position, top
from cjm_fasthtml_tailwind.core.base import combine_classes
from cjm_fasthtml_tailwind.utilities.transitions_and_animation import transition, duration

from cjm_fasthtml_settings.core.html_ids import HtmlIds

## Sidebar Menu Creation

In [ ]:
#| export
def create_sidebar_menu(
    schemas: Dict[str, Dict[str, Any]],  # Dictionary of schemas to display in sidebar
    active_schema: Optional[str] = None,  # The currently active schema name
    config_dir: Optional[Path] = None,  # Directory where config files are stored
    include_wrapper: bool = True,  # Whether to include the outer wrapper div
    menu_section_title: str = "Settings"  # Title for the settings section
) -> Div:  # Div or Ul element containing the sidebar menu
    """Create the sidebar navigation menu.
    
    Args:
        schemas: Dictionary mapping schema names to schema objects
        active_schema: Name of the currently active schema
        config_dir: Path to config directory (uses DEFAULT_CONFIG_DIR if None)
        include_wrapper: If False, returns just the Ul for OOB swaps
        menu_section_title: Title to display for the menu section
    
    Returns:
        Sidebar menu component
    """
    from cjm_fasthtml_settings import routes as settings_rt
    
    if config_dir is None:
        from cjm_fasthtml_settings.core.config import DEFAULT_CONFIG_DIR
        config_dir = DEFAULT_CONFIG_DIR
        
    menu_items = []

    # Add menu section title
    menu_items.append(
        Li(
            Span(menu_section_title, cls=str(menu_title))
        )
    )

    # Process settings schemas
    for key, schema in schemas.items():
        schema_name = schema.get("name")
        is_active = active_schema == schema_name
        is_configured = (Path(config_dir) / f"{schema_name}.json").exists()

        menu_items.append(
            Li(
                A(
                    Div(
                        Span(
                            schema.get("menu_title", schema.get("title")),
                            cls=str(font_weight.medium if is_active else "")
                        ),
                        Span(
                            "configured",
                            cls=combine_classes(badge, badge_colors.success, badge_sizes.xs, m.l(2))
                        ) if is_configured else None,
                        cls=combine_classes(flex_display, items.center, justify.between, w.full)
                    ),
                    href=settings_rt.index.to(id=schema_name),
                    hx_get=settings_rt.index.to(id=schema_name),
                    hx_target=HtmlIds.as_selector(HtmlIds.SETTINGS_CONTENT),
                    hx_push_url="true",
                    cls=combine_classes(
                        menu_modifiers.active if is_active else "",
                        transition.colors,
                        duration(200)
                    )
                ),
                id=HtmlIds.menu_item(schema_name)
            )
        )

    menu_ul = Ul(
        *menu_items,
        id=HtmlIds.SIDEBAR_MENU,
        cls=combine_classes(
            menu,
            bg_dui.base_200,
            w(64),
            p(4),
            min_h.full,
            border_radius.box
        )
    )

    if include_wrapper:
        return Div(
            menu_ul,
            cls=combine_classes(
                position.sticky,
                top(20),
                h.auto
            )
        )
    else:
        return menu_ul

In [ ]:
# Example: Create a sidebar menu
from cjm_fasthtml_settings.core.config import get_app_config_schema
from cjm_fasthtml_settings.core.schemas import SettingsRegistry

# Create registry with some schemas
registry = SettingsRegistry()
registry.register(get_app_config_schema(app_title="My App", include_theme=False))

# Custom schema
registry.register({
    "name": "custom",
    "title": "Custom Settings",
    "menu_title": "Custom",
    "type": "object",
    "properties": {}
})

sidebar = create_sidebar_menu(
    schemas=registry.get_all(),
    active_schema="general"
)

sidebar

```html
<div class="sticky top-20 h-auto">
  <ul id="sidebar-menu" class="menu bg-base-200 w-64 p-4 min-h-full rounded-box">
    <li>
<span class="menu-title">Settings</span>    </li>
    <li id="menu-item-general">
<a href="/settings/?id=general" hx-get="/settings/?id=general" hx-push-url="true" hx-target="#settings-content" class="menu-active transition-colors duration-200">        <div class="flex items-center justify-between w-full">
<span class="font-medium">Application</span>        </div>
</a>    </li>
    <li id="menu-item-custom">
<a href="/settings/?id=custom" hx-get="/settings/?id=custom" hx-push-url="true" hx-target="#settings-content" class="transition-colors duration-200">        <div class="flex items-center justify-between w-full">
<span>Custom</span>        </div>
</a>    </li>
  </ul>
</div>

```

## Helper: Create OOB Sidebar Menu

In [ ]:
#| export
def create_oob_sidebar_menu(
    schemas: Dict[str, Dict[str, Any]],  # Dictionary of schemas
    active_schema: str,  # Active schema name
    config_dir: Optional[Path] = None,  # Config directory
    menu_section_title: str = "Settings"  # Menu section title
):
    """Create sidebar menu with OOB swap attribute for HTMX.
    
    This is useful for updating the sidebar menu without a full page reload.
    """
    updated_menu = create_sidebar_menu(
        schemas=schemas,
        active_schema=active_schema,
        config_dir=config_dir,
        include_wrapper=False,
        menu_section_title=menu_section_title
    )
    updated_menu.attrs['hx-swap-oob'] = 'true'
    return updated_menu

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()